# Photoshop des Qualia : Démonstration

Ce notebook démontre le moteur d'oscillateurs de phase (Kuramoto/XY-Model) couplé à la détection de défauts topologiques.

**Objectif** : Simuler des états phénoménologiques distincts via des configurations de couplage différentes.

**Presets testés** :
1. **5-MeO-DMT** : Uniformisation progressive (annihilation des défauts)
2. **N,N-DMT** : Fragmentation chaotique (défauts persistants)

**Métriques observées** :
- Champ de phase θ(x,y)
- Nombre de défauts topologiques au fil du temps
- Paramètre d'ordre de Kuramoto r(t)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import json

from isinglab.oscillators import KuramotoXYEngine, MultiKernelConfig
from isinglab.analysis import detect_vortices, track_defects_over_time

%matplotlib inline


## Fonction utilitaire : Charger un preset


In [ ]:
def load_preset(preset_path):
    """Charge un preset JSON et retourne (engine, config_dict)."""
    with open(preset_path, 'r') as f:
        config_dict = json.load(f)
    
    # Parser kernel_config
    kc = config_dict['kernel_config']
    kernel_config = MultiKernelConfig(
        k1_strength=kc['k1_strength'],
        k1_range=kc['k1_range'],
        k1_sign=kc['k1_sign'],
        k2_strength=kc['k2_strength'],
        k2_range=kc['k2_range'],
        k2_sign=kc['k2_sign'],
        k3_strength=kc['k3_strength'],
        k3_range=kc['k3_range'],
        k3_sign=kc['k3_sign'],
        dt=kc['dt'],
        noise_amplitude=kc['noise_amplitude'],
        annealing_rate=kc['annealing_rate']
    )
    
    ec = config_dict['engine_config']
    engine = KuramotoXYEngine(
        shape=tuple(ec['grid_shape']),
        config=kernel_config,
        seed=ec['seed']
    )
    
    return engine, config_dict


## Fonction de simulation


In [ ]:
def run_simulation(engine, n_steps, record_interval=10):
    """Exécute la simulation et enregistre les états."""
    engine.reset()
    
    phase_history = []
    order_history = []
    
    for step in range(n_steps):
        engine.step()
        
        if step % record_interval == 0:
            phase_history.append(engine.get_phase_field())
            r, psi = engine.get_order_parameter()
            order_history.append(r)
    
    return phase_history, order_history


---

# Simulation 1 : 5-MeO-DMT (Uniformité)

Configuration :
- K1 fort (2.0), K2 = K3 = 0
- Annealing élevé (0.5)
- **Hypothèse** : Les défauts s'annihilent, r → 1


In [ ]:
engine_5meo, config_5meo = load_preset('../presets/pheno/5meo_uniformity.json')

print("Configuration 5-MeO-DMT :")
print(f"  Grid : {config_5meo['engine_config']['grid_shape']}")
print(f"  K1 : {config_5meo['kernel_config']['k1_strength']}")
print(f"  Annealing : {config_5meo['kernel_config']['annealing_rate']}")

n_steps = config_5meo['simulation']['n_steps']
record_interval = config_5meo['simulation']['record_interval']

print(f"\nExécution de {n_steps} steps...")
phase_history_5meo, order_history_5meo = run_simulation(engine_5meo, n_steps, record_interval)
print("✓ Simulation terminée")


In [ ]:
# Analyser les défauts
defect_metrics_5meo = track_defects_over_time(phase_history_5meo, threshold=0.5)
n_defects_5meo = [m.n_defects for m in defect_metrics_5meo]

print(f"Défauts initiaux : {n_defects_5meo[0]}")
print(f"Défauts finaux : {n_defects_5meo[-1]}")
print(f"Réduction : {n_defects_5meo[0] - n_defects_5meo[-1]} ({100*(1 - n_defects_5meo[-1]/(n_defects_5meo[0]+1)):.1f}%)")


In [ ]:
# Visualisation
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Phase fields à différents timesteps
timesteps = [0, len(phase_history_5meo)//2, len(phase_history_5meo)-1]
for i, t_idx in enumerate(timesteps):
    ax = axes[0, i]
    phase = phase_history_5meo[t_idx]
    im = ax.imshow(phase, cmap='hsv', vmin=0, vmax=2*np.pi, interpolation='bilinear')
    ax.set_title(f't = {t_idx * record_interval}')
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046)

# Métriques temporelles
ax_defects = axes[1, 0]
ax_defects.plot(n_defects_5meo, 'o-', color='red', label='Nombre de défauts')
ax_defects.set_xlabel('Frame')
ax_defects.set_ylabel('# Défauts')
ax_defects.set_title('Évolution des défauts')
ax_defects.grid(True, alpha=0.3)
ax_defects.legend()

ax_order = axes[1, 1]
ax_order.plot(order_history_5meo, 'o-', color='blue', label='r (ordre)')
ax_order.set_xlabel('Frame')
ax_order.set_ylabel('Paramètre d\'ordre r')
ax_order.set_title('Synchronisation globale')
ax_order.set_ylim([0, 1])
ax_order.grid(True, alpha=0.3)
ax_order.legend()

# Densité de défauts
ax_density = axes[1, 2]
density_5meo = [m.defect_density for m in defect_metrics_5meo]
ax_density.plot(density_5meo, 'o-', color='purple', label='Densité de défauts')
ax_density.set_xlabel('Frame')
ax_density.set_ylabel('Densité')
ax_density.set_title('Densité de défauts')
ax_density.grid(True, alpha=0.3)
ax_density.legend()

plt.tight_layout()
plt.suptitle('5-MeO-DMT : Uniformisation', fontsize=16, y=1.02)
plt.show()


---

# Simulation 2 : N,N-DMT (Fragmentation)

Configuration :
- K1 = 1.2 (positif), K2 = 0.8 (négatif), K3 = 0.3
- Pas d'annealing
- **Hypothèse** : Les défauts persistent, r reste faible


In [ ]:
engine_dmt, config_dmt = load_preset('../presets/pheno/dmt_fragmentation.json')

print("Configuration N,N-DMT :")
print(f"  Grid : {config_dmt['engine_config']['grid_shape']}")
print(f"  K1 : {config_dmt['kernel_config']['k1_strength']} (sign: {config_dmt['kernel_config']['k1_sign']})")
print(f"  K2 : {config_dmt['kernel_config']['k2_strength']} (sign: {config_dmt['kernel_config']['k2_sign']})")
print(f"  Annealing : {config_dmt['kernel_config']['annealing_rate']}")

n_steps_dmt = config_dmt['simulation']['n_steps']
record_interval_dmt = config_dmt['simulation']['record_interval']

print(f"\nExécution de {n_steps_dmt} steps...")
phase_history_dmt, order_history_dmt = run_simulation(engine_dmt, n_steps_dmt, record_interval_dmt)
print("✓ Simulation terminée")


In [ ]:
# Analyser les défauts
defect_metrics_dmt = track_defects_over_time(phase_history_dmt, threshold=0.5)
n_defects_dmt = [m.n_defects for m in defect_metrics_dmt]

print(f"Défauts initiaux : {n_defects_dmt[0]}")
print(f"Défauts finaux : {n_defects_dmt[-1]}")
print(f"Variation : {n_defects_dmt[-1] - n_defects_dmt[0]}")


In [ ]:
# Visualisation
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Phase fields à différents timesteps
timesteps_dmt = [0, len(phase_history_dmt)//2, len(phase_history_dmt)-1]
for i, t_idx in enumerate(timesteps_dmt):
    ax = axes[0, i]
    phase = phase_history_dmt[t_idx]
    im = ax.imshow(phase, cmap='hsv', vmin=0, vmax=2*np.pi, interpolation='bilinear')
    ax.set_title(f't = {t_idx * record_interval_dmt}')
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046)

# Métriques temporelles
ax_defects = axes[1, 0]
ax_defects.plot(n_defects_dmt, 'o-', color='red', label='Nombre de défauts')
ax_defects.set_xlabel('Frame')
ax_defects.set_ylabel('# Défauts')
ax_defects.set_title('Évolution des défauts')
ax_defects.grid(True, alpha=0.3)
ax_defects.legend()

ax_order = axes[1, 1]
ax_order.plot(order_history_dmt, 'o-', color='blue', label='r (ordre)')
ax_order.set_xlabel('Frame')
ax_order.set_ylabel('Paramètre d\'ordre r')
ax_order.set_title('Synchronisation globale')
ax_order.set_ylim([0, 1])
ax_order.grid(True, alpha=0.3)
ax_order.legend()

# Densité de défauts
ax_density = axes[1, 2]
density_dmt = [m.defect_density for m in defect_metrics_dmt]
ax_density.plot(density_dmt, 'o-', color='purple', label='Densité de défauts')
ax_density.set_xlabel('Frame')
ax_density.set_ylabel('Densité')
ax_density.set_title('Densité de défauts')
ax_density.grid(True, alpha=0.3)
ax_density.legend()

plt.tight_layout()
plt.suptitle('N,N-DMT : Fragmentation', fontsize=16, y=1.02)
plt.show()


---

# Comparaison directe

Comparons les deux régimes côte à côte.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 5-MeO-DMT
ax1 = axes[0]
ax1.plot(n_defects_5meo, 'o-', color='blue', label='5-MeO-DMT', linewidth=2)
ax1.set_xlabel('Frame', fontsize=12)
ax1.set_ylabel('Nombre de défauts', fontsize=12)
ax1.set_title('Annihilation des défauts (5-MeO)', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# N,N-DMT
ax2 = axes[1]
ax2.plot(n_defects_dmt, 'o-', color='red', label='N,N-DMT', linewidth=2)
ax2.set_xlabel('Frame', fontsize=12)
ax2.set_ylabel('Nombre de défauts', fontsize=12)
ax2.set_title('Persistance des défauts (DMT)', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

plt.tight_layout()
plt.show()


## Interprétation phénoménologique

**5-MeO-DMT (Uniformité)** :
- Les défauts topologiques s'annihilent rapidement
- Le paramètre d'ordre r augmente → synchronisation globale
- Interprétation : _Dissolution de la fragmentation perceptuelle, uniformité consciente_

**N,N-DMT (Fragmentation)** :
- Les défauts persistent malgré l'évolution temporelle
- r reste faible → pas de synchronisation globale
- Interprétation : _Multi-domaines perceptuels stables, chaos organisé_

---

## Prochaines étapes

1. **Contrôle holonomique** : Implémenter les strokes définis dans `isinglab/control/holonomy.py`
2. **Projection 3D** : Intégrer `ProjectionMap` pour moduler spatialement les kernels
3. **Phase de Berry** : Calculer la phase géométrique le long des trajectoires fermées
4. **Atlas Bridge** : Connecter aux paramètres physiques de l'Atlas (température, pression, champs EM)

**Status** : Moteur de base ✓ | Défauts topologiques ✓ | Contrôle holonomique (structure) ✓
